In [1]:
from base_vit import ViT
import torch
from lora import LoRA_ViT
import numpy as np
import matplotlib.pyplot as plt
from utils.dataloader import _kneeDataloader
from tqdm import tqdm
import time

In [2]:
model = ViT('B_16_imagenet1k')
model.load_state_dict(torch.load('B_16_imagenet1k.pth'))
model.eval()
criterion = torch.nn.CrossEntropyLoss()

In [3]:
weights = model.state_dict()
trainable_keys = [k for k, v in weights.items() if v.requires_grad]
trainable_weights = {k: v.clone() for k, v in weights.items() if k in trainable_keys}

In [4]:
grid_size = 10
delta = 0.1
trainloader = _kneeDataloader(bs=16)[1]
loss_grid = np.zeros((grid_size, grid_size))
for i in range(grid_size):
    for j in range(grid_size):
        start = time.time()
        for k, v in weights.items():
            weights[k] = weights[k] + delta * (i - grid_size // 2) * v + delta * (
                j - grid_size // 2) * torch.randn_like(v)
        model.load_state_dict(weights)
        print('change weight takes',time.time()-start)
        running_loss = 0.0
        for _, data in tqdm(enumerate(trainloader, 0)):
            inputs, labels = data

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()

        loss_grid[i, j] = running_loss
        print('all_process',time.time()-start)


change weight takes 1.0217909812927246


2it [01:47, 53.85s/it]


KeyboardInterrupt: 

In [ ]:
plt.imshow(loss_grid, cmap='jet', interpolation='nearest')
plt.colorbar()
plt.show()